In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('order_brush_order.csv')

In [3]:
df.head()
res = dict()
# res shape should be 18770 * 2
remain_answer_shop = 18770

## Get unique shop id 

In [4]:
print(df.shape)
print("number of shop id: ", df['shopid'].nunique())
print("number of user id: ", df['userid'].nunique())
remain_answer_shop = df['shopid'].nunique()
unique_shopId = df['shopid'].unique()
unique_userId = df['userid'].unique()
print("unique shop id: ", unique_shopId, len(unique_shopId))
print("unique user id: ", unique_userId, len(unique_userId))

(222750, 4)
number of shop id:  18770
number of user id:  93883
unique shop id:  [ 93950878 156423439 173699291 ...  50236030 203587596 147118226] 18770
unique user id:  [30530270 46057927 67341739 ... 55114470 41654255 27306783] 93883


## Filtered by buyers less than 3

In [5]:
group_shop = df.groupby(['shopid'])
filter_shopId = []
for i, shopId in enumerate(unique_shopId):
    if group_shop.get_group(shopId).shape[0] < 3:
        res[shopId] = 0
        filter_shopId.append(shopId)
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  9739 remain answers:  9031 True


## Filter each shop by count number of each userid which less than 2

In [6]:
# sort event time in df
freqCount = 0
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid).sort_values(["event_time"])
    userfreq = testdf['userid'].value_counts()
    if userfreq.max() >= 3:
        freqCount += 1
    else:
        res[uid] = 0
        filter_shopId.append(uid)
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  17942 remain answers:  828 True


## Get user dataframe from specific shop dataframe

In [7]:
testdf = group_shop.get_group(unique_shopId[0]).sort_values(["event_time"])
userfreq = testdf['userid'].value_counts()
numberCandicate, candicates = userfreq[userfreq >=3].index.shape[0], np.array(userfreq[userfreq >=3].index)

subtestdf = testdf[testdf['userid'] == candicates[0]].sort_values(["event_time"])
print("number of candicate user id: ", numberCandicate)
print("candicate userids: ",candicates)
print(subtestdf)

number of candicate user id:  1
candicate userids:  [110240400]
              orderid     shopid     userid           event_time
43418  31280563745641  173699291  110240400  2019-12-29 09:02:43
30408  31387219757278  173699291  110240400  2019-12-30 14:40:19
49832  31387324931487  173699291  110240400  2019-12-30 14:42:05


### Note: How to calculate datetime

In [8]:
import datetime
from dateutil.parser import parse
now = datetime.datetime.now()
print(now)
a = parse('January 31, 2010')
print(a)
td = datetime.timedelta(weeks=1, days=30, hours=2, minutes=40)
print(td)
t1 = parse(group_shop.get_group(unique_shopId[0]).sort_values(["event_time"]).iloc[0]['event_time'])
t2 = parse(group_shop.get_group(unique_shopId[0]).sort_values(["event_time"]).iloc[1]['event_time'])
oneHour = datetime.timedelta(hours=1)
print(t1)
print(t2)
print(t2-t1)

print(oneHour)
print((t2-t1) < oneHour)

2021-03-01 17:18:32.890807
2010-01-31 00:00:00
37 days, 2:40:00
2019-12-27 00:48:28
2019-12-27 01:00:48
0:12:20
1:00:00
True


## Get candicates which equal to 3 and difference between start and end time >one hour

In [9]:
oneHour = datetime.timedelta(hours=1)
subTables = []
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid)
    userfreq = testdf['userid'].value_counts()
    numberCandicate = userfreq[userfreq >=3].index.shape[0]
    candicates = np.array(userfreq[userfreq >=3].index)
    
    if(numberCandicate == 1):
        subtestdf = testdf[testdf['userid'] == candicates[0]].sort_values(["event_time"])
        if(subtestdf.shape[0] == 3):
            startT = parse(subtestdf.iloc[0]['event_time'])
            endT = parse(subtestdf.iloc[-1]['event_time'])
            if((endT-startT) > oneHour):
                res[uid] = 0
                filter_shopId.append(uid)

    
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  18255 remain answers:  515 True


## Filter candicates which equal to 3 and difference between start and end time >one hour

### Condition: number of candicate == 0,  

In [10]:
subTables = []
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid)
    userfreq = testdf['userid'].value_counts()
    candicates = np.array(userfreq[userfreq >=3].index)
    filter_candicates = []
    
    for user in candicates:
        subtestdf = testdf[testdf['userid'] == user].sort_values(["event_time"])
        if(subtestdf.shape[0] == 3):
            startT = parse(subtestdf.iloc[0]['event_time'])
            endT = parse(subtestdf.iloc[-1]['event_time'])
            if((endT-startT) > oneHour):
                filter_candicates.append(user)
    candicates = [c for c in candicates if c not in filter_candicates]
    if len(candicates) == 0:
        res[uid] = 0
        filter_shopId.append(uid)
    else:
        subTables.append({'shopid': uid,
                     "candicates": candicates})

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  18281 remain answers:  489 True


In [11]:
subTables = sorted(subTables, key=lambda k: len(k['candicates'])) 
print(len(subTables))

489


## The column index (start time and end time) is continue, order brushing

In [12]:
filter_shopId = []
count = 0
for s in subTables:
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if(len(s['candicates']) == 1):
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        startT = subtestdf.iloc[0]['time']
        endT = subtestdf.iloc[-1]['time']
        if ((endT-startT) < oneHour):
            startIdx = testdf[testdf['time'] == startT].index.values[0]
            endIdx = testdf[testdf['time'] == endT].index.values[0]
            if((endIdx - startIdx + 1) == subtestdf.shape[0]):
                res[s['shopid']] = s['candicates'][0]
                filter_shopId.append(s['shopid'])
                count += 1
                

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)
subTables = [t for t in subTables if t["shopid"] not in filter_shopId]
print(len(subTables))

Got number of answers:  18542 remain answers:  228 True
228


In [13]:
filter_shopId = []
for s in subTables:
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if(len(s['candicates']) == 1):
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        startT = subtestdf.iloc[0]['time']
        endT = subtestdf.iloc[-1]['time']
        if ((endT-startT) < oneHour):
            startIdx = testdf[testdf['time'] == startT].index.values[0]
            endIdx = testdf[testdf['time'] == endT].index.values[0]
#             print(testdf.loc[startIdx:endIdx][["userid", "time"]])
            subTestdf = testdf.loc[startIdx:endIdx]
            uniqueUser = subTestdf['userid'].nunique()
            numOders = subTestdf.shape[0]
            isOderBrushing = (numOders/uniqueUser) >=3
            if isOderBrushing:
                res[s['shopid']] = s['candicates'][0]
                filter_shopId.append(s['shopid'])
            else:
                res[s['shopid']] = 0
                filter_shopId.append(s['shopid'])
#             print(subtestdf.shape[0], startIdx, endIdx, uniqueUser, numOders,isOderBrushing, numOders/uniqueUser)

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)
subTables = [t for t in subTables if t["shopid"] not in filter_shopId]
print(len(subTables))

Got number of answers:  18577 remain answers:  193 True
193


In [42]:
def process_unit(subtestdf, kmeans):
    subtestdf['label'] = kmeans.fit_predict(subtestdf[['time']])
            
    keys = subtestdf['label'].value_counts().keys().tolist()
    values = subtestdf['label'].value_counts().tolist()
    labels_count = dictionary = dict(zip(keys, values))
    for k, v in zip(keys, values):
        if v >= 3 :
            sub = subtestdf[subtestdf['label'] == k]
            startT = sub.iloc[0]['time']
            endT = sub.iloc[-1]['time']
            if ((endT-startT) < oneHour):
                pass
            else:
                sub['label'] = kmeans.fit_predict(sub[['time']])
                process_unit(sub, kmeans)
                print(sub)

In [45]:
import time
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import kde
from matplotlib.pyplot import plot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans

maxN = 0
for s in subTables:
    
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if len(s['candicates']) == 1:
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        
        subtestdf.drop(['orderid', 'shopid', 'userid', 'event_time'], axis = 1, inplace=True)
        maxN = subtestdf.shape[0] if maxN < subtestdf.shape[0] else maxN
        if maxN == 23:
            kmeans = KMeans(n_clusters=2)
            process_unit(subtestdf, kmeans)
        
print(maxN)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

                  time  label
53 2019-12-31 15:14:17      0
54 2019-12-31 15:15:40      0
55 2019-12-31 15:18:29      0
56 2019-12-31 15:21:36      0
57 2019-12-31 15:27:55      0
58 2019-12-31 15:31:38      0
59 2019-12-31 15:37:25      0
62 2019-12-31 18:46:27      1
                  time  label
50 2019-12-31 10:25:49      1
53 2019-12-31 15:14:17      0
54 2019-12-31 15:15:40      0
55 2019-12-31 15:18:29      0
56 2019-12-31 15:21:36      0
57 2019-12-31 15:27:55      0
58 2019-12-31 15:31:38      0
59 2019-12-31 15:37:25      0
62 2019-12-31 18:46:27      0
                  time  label
30 2019-12-30 16:37:02      0
31 2019-12-30 16:48:36      0
32 2019-12-30 16:52:27      0
33 2019-12-30 16:56:40      0
34 2019-12-30 16:59:57      0
35 2019-12-30 17:16:42      0
36 2019-12-30 17:21:10      0
37 2019-12-30 17:26:28      0
38 2019-12-30 17:30:36      0
39 2019-12-30 17:35:45      0
50 2019-12-31 10:25:49      1
53 2019-12-31 15:14:17      1
54 2019-12-31 15:15:40      1
55 2019-12

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

                  time  label
13 2019-12-30 01:35:40      1
20 2019-12-31 01:21:12      0
21 2019-12-31 01:25:05      0
                  time  label
46 2019-12-30 00:59:53      0
47 2019-12-30 01:07:28      0
70 2019-12-31 17:50:00      1
                  time  label
58 2019-12-30 10:13:59      0
59 2019-12-30 10:23:15      0
62 2019-12-30 17:29:03      1
                  time  label
58 2019-12-30 10:13:59      1
59 2019-12-30 10:23:15      1
62 2019-12-30 17:29:03      1
75 2019-12-31 10:23:57      0
                  time  label
54 2019-12-30 11:29:12      1
58 2019-12-30 15:57:07      0
59 2019-12-30 16:05:46      0
                  time  label
54 2019-12-30 11:29:12      0
58 2019-12-30 15:57:07      0
59 2019-12-30 16:05:46      0
63 2019-12-30 20:06:19      1
64 2019-12-30 20:09:34      1
                  time  label
54 2019-12-30 11:29:12      0
58 2019-12-30 15:57:07      0
59 2019-12-30 16:05:46      0
63 2019-12-30 20:06:19      0
64 2019-12-30 20:09:34      0
76 2019-12

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

...

                 time  label
0 2019-12-27 23:36:49      0
1 2019-12-27 23:38:01      0
2 2019-12-28 00:04:55      0
3 2019-12-28 00:06:40      0
4 2019-12-28 00:34:04      1
5 2019-12-28 00:37:09      1
                 time  label
0 2019-12-27 23:36:49      0
1 2019-12-27 23:38:01      0
2 2019-12-28 00:04:55      0
3 2019-12-28 00:06:40      0
4 2019-12-28 00:34:04      0
5 2019-12-28 00:37:09      0
7 2019-12-28 14:51:18      1
                 time  label
6 2019-12-30 14:11:54      0
7 2019-12-30 16:12:59      1
8 2019-12-30 17:40:50      1
                 time  label
2 2019-12-30 00:04:56      1
3 2019-12-30 00:07:09      1
4 2019-12-31 01:59:01      0
                 time  label
1 2019-12-27 00:38:32      0
4 2019-12-28 20:52:59      1
5 2019-12-28 20:55:02      1


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

                  time  label
9  2019-12-29 23:58:48      1
10 2019-12-30 00:07:21      1
13 2019-12-30 14:28:04      0
14 2019-12-30 14:28:32      0
                  time  label
7  2019-12-29 12:47:45      1
8  2019-12-29 13:03:33      1
10 2019-12-29 16:50:04      0
                 time  label
0 2019-12-27 12:09:04      0
2 2019-12-27 16:17:12      0
3 2019-12-28 10:15:23      1
                 time  label
0 2019-12-28 17:47:40      1
1 2019-12-28 17:48:48      1
2 2019-12-28 19:29:58      0


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


23
